In [1]:
import numpy as np
from PIL import Image

In [2]:
def lut_256(colors: np.ndarray) -> np.ndarray:
    cshp = colors.shape
    assert cshp[0] > 1
    assert cshp[1] == 3
    
    ypal = np.zeros((256, 3), dtype=np.float)
    xpal = np.linspace(0.0, 1.0, 256, dtype=np.float)
    xcol = np.linspace(0.0, 1.0, cshp[0], dtype=np.float)

    for i in range(0, 3):
        ycol = colors[:, i] / 255.0
        ypal[:, i] = np.power(np.interp(xpal, xcol, ycol), 2.2)

    return ypal

In [3]:
def export_lut_12bits(lut: np.ndarray) -> str:
    tmp = lut * 15.0 + 0.5
    tmp = tmp.astype(np.uint16)
    lut12 = tmp[:, 0] << 8 | tmp[:, 1] << 4 | tmp[:, 2]
    
    header = "#pragma once\n"
    header += "const uint16_t lut[] = {"
    for i in range(0, 256):    
        header += hex(lut12[i]) + ", "
    header += "};\n"
    
    return header

In [4]:
def sine_lut_256() -> np.ndarray:
    xtable = np.linspace(0.0, 2.0*np.pi, 256, endpoint=False)
    ytable = np.sin(xtable)
    return ytable

In [5]:
def export_lut_16bits(lut: np.ndarray) -> str:
    tmp = lut * 255.0 + 0.5
    lut8 = tmp.astype(np.int16)
    
    header = "#pragma once\n"
    header += "const int16_t sine[] = {"
    for i in range(0, 256):
        header += hex(lut8[i]) + ', '
    header += "};\n"
    
    return header

In [6]:
def export_hmap_8bits(hmap: np.ndarray) -> str:
    header = "#pragma once\n"
    header += "const uint8_t hmap[] = {\n"
    for j in range(0, hmap.shape[0]):
        for i in range(0, hmap.shape[1]):
            header += hex(hmap[j, i]) + ", "
        header += "\n"
    header += "};\n"
    return header

In [14]:
#colors = np.array([[0xFF, 0x00, 0x00], [0xFF, 0x7F, 0x00], [0xFF, 0xFF, 0x00],
#                   [0x00, 0xFF, 0x00], [0x00, 0x00, 0xFF], [0x4B, 0x00, 0x82],
#                   [0x94, 0x00, 0xD3], [0xFF, 0x00, 0x00]], dtype=np.uint8)
#colors = np.array([[0x00, 0x00, 0x00], [0xFF, 0x00, 0x00], [0xFF, 0xFF, 0x00],
#                   [0x00, 0x00, 0xFF], [0x00, 0xFF, 0xFF]], dtype=np.uint8)
#colors = np.array([[0x00, 0xBF, 0xFF], [0x00, 0xFF, 0x00], [0x00, 0x7F, 0x00], [0xB8, 0x86, 0x0B],
#                   [0x8B, 0x45, 0x13], [0x80, 0x80, 0x80], [0xFF, 0xFF, 0xFF]], dtype=np.uint8)
#colors = np.array([[0x00, 0x00, 0x3F], [0x00, 0x00, 0xFF], [0xFF, 0xFF, 0xFF],
#                   [0xFF, 0x00, 0x00], [0x3F, 0x00, 0x00]], dtype=np.uint8)
#colors = np.array([[0xFF, 0xFF, 0x00], [0xFF, 0x00, 0x00], [0xFF, 0xFF, 0x00]], dtype=np.uint8)
#colors = np.array([[0x00, 0xFF, 0xFF], [0x00, 0xFF, 0x00], [0x00, 0xFF, 0xFF]], dtype=np.uint8)
#colors = np.array([[0xFF, 0x00, 0xFF], [0x00, 0x00, 0xFF], [0xFF, 0x00, 0xFF]], dtype=np.uint8)
#colors = np.array([[0xFF, 0xFF, 0x00], [0x00, 0xFF, 0xFF], [0xFF, 0xFF, 0x00]], dtype=np.uint8)
#colors = np.array([[0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF],
#                   [0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF],
#                   [0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF],
#                   [0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF]], dtype=np.uint8)
#colors = np.array([[0x00, 0x00, 0x00], [0x00, 0x00, 0x00],
#                   [0x00, 0x00, 0x00], [0x00, 0x00, 0x00],
#                   [0x00, 0x00, 0x00], [0x00, 0x00, 0x00],
#                   [0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF]], dtype=np.uint8)
#colors = np.array([[0x00, 0x00, 0x00], [0x00, 0x00, 0x00],
#                   [0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF],
#                   [0x00, 0x00, 0x00], [0x00, 0x00, 0x00],
#                   [0x00, 0x00, 0x00], [0x00, 0x00, 0x00]], dtype=np.uint8)
colors = np.array([[0x00, 0x00, 0x00], [0xFF, 0xFF, 0xFF]], dtype=np.uint8)

pal = lut_256(colors)
header = export_lut_12bits(pal)
with open("gen.h", "w") as f:
    f.write(header)

In [10]:
lut = sine_lut_256()
header = export_lut_16bits(lut)
with open("sine.h", "w") as f:
    f.write(header)

In [47]:
hmap = Image.open("heightmap.png")
img = hmap.resize((128, 128), resample=Image.BOX)
tmp = np.array(img)

tshape = (tmp.shape[0]*2, tmp.shape[1]*2)
tile = np.zeros((tshape[0], tshape[1]))
tile[0:tmp.shape[0], 0:tmp.shape[1]] = tmp
tile[0:tmp.shape[0], tmp.shape[1]:tshape[1]] = tmp[:, ::-1]
tile[tmp.shape[0]:tshape[0], 0:tmp.shape[1]] = tmp[::-1, :]
tile[tmp.shape[0]:tshape[0], tmp.shape[1]:tshape[1]] = tmp[::-1, ::-1]

bias = 1.2 # create more lakes
tile = (tile - bias*np.min(tile)) / (np.max(tile) - bias*np.min(tile)) * 255.0
tile = np.clip(tile, 0, 255)

header = export_hmap_8bits(tile.astype(np.uint8))
with open("hmap.h", "w") as f:
    f.write(header)